In [44]:
import json
import os
import glob
import numpy as np
import keras as kr
from datetime import datetime
from IPython.display import clear_output

In [45]:
def load_dictionaries():
    text = ' hello world'
    char_id = dict(zip(set(text), range(len(set(text)))))
    id_char = dict(zip(char_id.values(), char_id.keys()))
    return id_char, char_id

def get_article_list():
    os.chdir('cleaned')
    file_list = glob.glob('*.txt')
    os.chdir('..')
    return file_list

def get_article(name):
    os.chdir('cleaned')
    with open(name, 'r') as f:
        text = f.read()
    os.chdir('..')
    return ' hello world ' * 500
    return text

id_char, char_id = load_dictionaries()
article_list = get_article_list()

In [46]:
def convert_char_id(text):
    result = []
    features = len(char_id)
    for i, char in enumerate(text):
        position = int(char_id[char])
        observation = np.zeros(( features))
        observation[position] = 1
        result.append(observation)
    return np.array(result)

def convert_id_char(onehot):
    result = ''
    for letter in onehot:
        l = list(letter).index(max(letter))
        result = result + id_char[l]
    return result

def get_X_y(list_of_one_hots, sentence_length):
    y = []
    X = []
    for i, letter in enumerate(list_of_one_hots):
        if i > sentence_length:
            y.append(letter)

            x = list_of_one_hots[i- sentence_length:i]
            X.append(x)

    return np.array(X), np.array(y)

In [47]:
def predict(text):
    onehot = convert_char_id(text[-sentence_length:])
    return convert_id_char(model.predict(np.array([onehot]), 1))

In [48]:
def train_on_article(name, epochs):
    global model
    #text = get_article(name)
    text = 'hello world ' * 10
    onehot = convert_char_id(text)
    X, y = get_X_y(onehot,sentence_length )
    if type(model) == type(None):
        model = generate_model(X, y)
    model.fit(X, y, epochs = epochs, batch_size= 100)

In [49]:
def generate_model(X, y):
    model = kr.models.Sequential()
    model.add(kr.layers.LSTM(16, input_shape=X.shape[1:],return_sequences= True ))
    model.add(kr.layers.Dropout(0.1))
    model.add(kr.layers.LSTM(16))
    model.add(kr.layers.Dense(X.shape[2]))
    model.add(kr.layers.Activation('softmax'))
    optimizer = kr.optimizers.Adam()
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

model = None

In [50]:
def print_letters(seed_text, length):
    while len(seed_text) < sentence_length:
        seed_text = ' ' + seed_text
    
    result = ''
    last_val = ''
    for num in range(length):
        last_val = predict(seed_text)
        result = result + last_val
    return result

In [51]:
sentence_length= 10

In [52]:
def log(message):
    print(message)
    with open('sanity_log.txt', 'a') as f:
        f.write('\n')
        f.write(datetime.now().isoformat())
        f.write('    :')
        f.write(message)
    return 

In [53]:
for article in article_list:
    train_on_article(article, 100)
    clear_output(True)
    log(print_letters('hello world', 500))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
Epoch 1/100
109/109 [==============================] - 0s 174us/step - loss: 0.0013
Epoch 2/100
109/109 [==============================] - 0s 184us/step - loss: 0.0013
Epoch 3/100
109/109 [==============================] - 0s 174us/step - loss: 0.0013
Epoch 4/100
109/109 [==============================] - 0s 174us/step - loss: 0.0015
Epoch 5/100
109/109 [==============================] - 0s 184us/step - loss: 0.0013
Epoch 6/100
109/109 [==============================] - 0s 184us/step - loss: 0.

KeyboardInterrupt: 